### Samples of fetching Product View with Mark2Market

This first section loads a token from a local .env file 

In [3]:
# Import relevant libraries
import sys,os
from os.path import join, dirname
from dotenv import load_dotenv

sys.path.insert(1, os.path.join(sys.path[0], '..'))  # Hack to include parent directory in search path

# Loading token, url etc from .env file in current directory
load_dotenv('.env')
token = os.environ.get("ENERGYDESK_TOKEN")
server_url = os.environ.get("ENERGYDESK_URL")
print("Using server ", server_url)

Using server  https://celsio-test.energydesk.no/appserver


### Initialize the Python SDK which wraps sme useful REST calls ###

In [4]:
from IPython.display import display, HTML
from energydeskapi.sdk.common_utils import init_api_from_environment
from energydeskapi.customers.customers_api import CustomersApi
from energydeskapi.customers.users_api import UsersApi
from energydeskapi.portfolios.portfoliotree_api import PortfolioTreeApi, PortfoliosApi
from energydeskapi.types.company_enum_types import CompanyTypeEnum, CompanyRoleEnum
import pandas as pd
import json
api_connection=init_api_from_environment()
print(api_connection.token)
jsdata=PortfoliosApi.get_portfolios_embedded(api_connection)
df=pd.DataFrame(data=jsdata)
display(HTML(df[['pk', 'description']].to_html()))


5c287d277ba3b8b45903133683eb0d2f7a1b889e


,pk,description
0,1,Sikring Old
1,2,Dynamic
2,3,Sikring Kraft
3,4,Elektrisk Forbruk
4,5,Brensler Varmeproduksjon
5,6,Varmesalg
6,7,Salg av Produsert kraft
7,8,Forbruk
8,9,Salg
9,10,Celsio Energy Total


## Calculate Portfolio ##

In [5]:
from energydeskapi.portfolios.portfolioviews_api import PortfolioViewsApi
import pandas as pd
import pendulum
from IPython.core.display import display_html
from matplotlib import pyplot as plt
from energydeskapi.settlement.settlement_api import SettlementApi

def __extract_rename_pandas_cols(dframe, **kwargs):
    keys = sorted(kwargs.keys(), key=lambda x: list(dframe).index(x))
    return dframe.filter(items=keys).rename(columns=kwargs)

def __column_aggregation():
    colummap = {'netvol': 'sum', 'buyvol': 'sum', 'sellvol': 'sum',
                'avgcost': 'mean', 'avgcostbuy': 'mean', 'avgcostsell': 'mean',
                'delivery_from': 'max', 'delivery_until': 'max',
                'netpos': 'mean', 'buypos': 'mean', 'sellpos': 'mean', 'spot': 'mean', 'curve': 'mean','unrealized': 'sum','realized': 'sum'}
    return colummap

grp=["trade_id"]
filter={
    'view_currency': "NOK",
    'portfolio_id':13,  # See list in previous section
    'groupby__in': grp,
    "view_period_from__gte":"2024-01-01",
    "view_period_until__lt": "2026-01-01",
    }

v, df = SettlementApi.get_product_result_view_df(api_connection, filter)
df = df.groupby("ticker").agg(__column_aggregation())
df=df.reset_index()
print("All columns: ", list(df.columns))
df=__extract_rename_pandas_cols(df, ticker="Product", netpos="MW", spot="Spot", curve="Fwd Curve",unrealized="Urealisert", realized="Levert")
display_html(df.to_html(),raw=True)


https://celsio-test.energydesk.no/appserver/api/settlement/settlementview/productresults/?view_currency=NOK&portfolio_id=13&groupby__in=trade_id&view_period_from__gte=2024-01-01&view_period_until__lt=2026-01-01
All columns:  ['ticker', 'netvol', 'buyvol', 'sellvol', 'avgcost', 'avgcostbuy', 'avgcostsell', 'delivery_from', 'delivery_until', 'netpos', 'buypos', 'sellpos', 'spot', 'curve', 'unrealized', 'realized']


/opt/homebrew/lib/python3.10/site-packages/energydeskapi/settlement/settlement_api.py:111: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_res, orient="table")


,Product,MW,Spot,Fwd Curve,Urealisert,Levert
0,ENOAFUTBLMAPR-24,-4.291667,573.931458,0.000000,0.0,-1.171973e+06
1,ENOAFUTBLMFEB-24,-34.500000,644.040000,0.000000,0.0,1.541378e+07
2,ENOAFUTBLMJAN-24,-34.500000,644.040000,0.000000,0.0,1.899227e+07
3,ENOAFUTBLMJUN-24,-14.000000,0.000000,0.000000,0.0,1.207584e+07
4,ENOAFUTBLMMAR-24,-4.257143,640.606756,0.000000,0.0,6.163358e+06
5,ENOAFUTBLMMAY-24,-2.588235,0.000000,0.000000,0.0,2.955733e+07
6,ENOFUTBLQ1-25,-1.625000,0.000000,649.125509,0.0,1.868668e+07
7,ENOFUTBLQ3-24,-14.000000,0.000000,0.000000,0.0,2.676979e+07
8,ENOFUTBLQ4-24,-14.000000,0.000000,0.000000,0.0,3.853380e+07
9,ENOFUTBLYR-25,-5.000000,0.000000,452.649743,0.0,4.502640e+07


In [12]:
from pivottablejs import pivot_ui
from IPython.display import display,IFrame, HTML
display(HTML('<section><h1>Hello</h1><p>Text</p></section>'))
pivot_ui(df, outfile_path='pivottablejs.html')
display(HTML(filename='pivottablejs.html'))

from pivottablejs import pivot_ui

pivot_ui(df,      
    cols= ["Party"],
    rows= ["Province"],
    rendererName= "Horizontal Stacked Bar Chart",
    rowOrder= "value_z_to_a", 
    colOrder= "value_z_to_a",
    rendererOptions= {
        "c3": { "data": {"colors": {
            "Liberal": '#dc3912', "Conservative": '#3366cc', "NDP": '#ff9900',
            "Green":'#109618', 'Bloc Quebecois': '#990099'
        }}}
    }
)
import tabloo

tabloo.show(df)

 * Serving Flask app 'tabloo.server'
 * Debug mode: off
